In [1]:
#### import ssdeep
import pickle,os,sys,gc
import pandas as pd
# pd.set_option('display.max_columns', None)
import numpy as np
from tqdm import tqdm
from collections import Counter
import multiprocessing as mp
from multiprocessing import Process, Manager, Pool
import subprocess
from multiprocessing import Pool 
import time,datetime
from itertools import chain
import math
from operator import itemgetter
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import functools
import itertools
import operator
from multiprocessing.dummy import Pool as ThreadPool
import os.path
from os import path
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

* 拉出payload pkt timelist、算出統計
* 須從session table對應回去本來的session timestamp

In [ ]:
protocols_need = ['ssh','smb','http','telnet','smtp','ftp','sip','mysql','rpc','pop','imap']
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
try:
    protos_pktinfo_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict_sampled_pktinfo.pkl','rb'))
except :
    protos_pktinfo_df = None
    tries=1
    while(protos_pktinfo_df is None):
        tries+=1
        print(tries)
        try:
            gc.collect()
            protos_pktinfo_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict_sampled_pktinfo.pkl','rb'))
        except:
            protos_pktinfo_df = None
    
protos_pktinfo_df['http']

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [2]:
# read AP table (all protocols)
agg_proto_df_dict = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict.pkl','rb'))
protocols_need = ['ssh','smb','http','telnet','smtp','ftp','sip','mysql','rpc','pop','imap']
agg_proto_df_dict['http']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,date_count,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num
0,20200108_8589990159_http,"[1578491422.648805, 1578491434.488182, 1578491...","[South Korea, South Korea, South Korea, South ...","[122.41.87.188, 122.41.87.188, 122.41.87.188, ...",[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"[(37.5112, 126.9741), (37.5112, 126.9741), (37...",亞太電信,http,118,[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"{'48': ['a', 'q', 'F6GY18n', 'R', 'M', '5Y1k']}",[48:Ga7UCqF6GY18nwiLkcRq09jcRq09uMRRdJtYMY8:hg...,"[1578491500.667118, 1578491500.667435, 1578491...","{'20200108': 7, '20200109': 111}",0.0,"{'亞太電信_20200108': 7, '亞太電信_20200109': 111}",21036,118,3,3
1,20200106_8589962872_http,"[1578244875.871224, 1578279973.587717, 1578291...","[Netherlands, Netherlands, Netherlands, Nether...","[51.15.71.134, 51.15.71.134, 51.15.71.134, 51....",[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,"[(52.3759, 4.8975), (52.3759, 4.8975), (52.375...",亞太電信,http,25,[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,{'12': ['Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQo...,[12:Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQom:Lep...,"[1578315077.277642, 1578315077.277914, 1578315...","{'20200106': 5, '20200107': 5, '20200108': 2, ...",0.0,"{'亞太電信_20200106': 5, '亞太電信_20200107': 5, '亞太電信...",5434,25,1,1
2,20200106_206158498684_http,"[1578325476.581104, 1578325480.685079]","[Canada, Canada]","[209.99.129.253, 209.99.129.253]",[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[(43.6655, -79.4204), (43.6655, -79.4204)]",亞太電信,http,2,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,{'96': ['rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41u...,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[1578325476.581104, 1578325476.581368, 1578325...",{'20200106': 2},0.0,{'亞太電信_20200106': 2},441,2,1,1
3,20200109_128849077624_http,"[1578544003.66182, 1578544004.619287]","[United Kingdom, United Kingdom]","[37.220.22.180, 88.150.210.33]",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[(53.552, -1.4872), (51.1912, -0.3432)]",亞太電信,http,2,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"{'48': ['a7U', '4CyGXU', 'Y18n', 'b', 'W:', 'd...",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[1578544004.619287, 1578544004.619586, 1578544...",{'20200109': 2},0.0,{'亞太電信_20200109': 2},346,2,2,1
4,20200106_534_http,"[1578244070.453082, 1578248759.696275, 1578252...","[China, France, Thailand, Thailand, China, Ban...","[119.52.109.64, 83.206.128.205, 146.88.51.234,...",[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"[(42.5272, 125.6753), (48.7142, 7.1898), (13.7...",中華電信,http,36,[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"{'96': ['nXF', 'Y', '3I', 'nXFI', 'I', '4', '4...",[96:TnXFUYU33ImnXFI1I4PUmlXnXNUm4dUm8q3UUKnXFU...,"[1578295954.030631, 1578295954.030895, 1578295...",{'20200106': 36},0.0,{'中華電信_20200106': 36},31210,36,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,"[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"{'48': ['a', 'Q5jqXMz', 'A', 'q', ':', '5Rz', ...",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854...","{'20200109': 8, '20200110': 3}",0.0,"{'台灣碩網_20200109': 8, '台灣碩網_20200110': 3}",15332,11,2,2
340,20200107_206158449329_http_20200106_1202590851...,"[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386,

In [3]:
def left_most_keyword(left_df):
    allproto_isp_da = []
    for i,proto in enumerate(protocols_need):    
        proto_df = left_df[proto]
        if len(proto_df)<1:
            continue
        all_isp_da = proto_df['isp_date_sessioncount'].tolist()
    #     if i==0:
        all_isp_da = [list(x.keys()) for x in all_isp_da]
    #     else:
        allproto_isp_da.extend(all_isp_da)
    #     break
    allproto_isp_da = list(functools.reduce(operator.iconcat, allproto_isp_da, []))
    count_di = dict(Counter(allproto_isp_da))
    count_di = dict(sorted(count_di.items(),key=lambda item:item[1],reverse=True))
    j=0
    while(True):
        try:
            word =  list(count_di.keys())[j]
            if '遠傳' in word:
                j+=1
            else:
                return word,j,count_di
        except IndexError:
            return 'DONE',j,count_di
#     return word

In [4]:
def is_intaken(di,keyword):
    if keyword in di.keys():
        return True
    else:
        return False
    
    
left_proto_df_dict = agg_proto_df_dict.copy()
k=0
words = []
while(True):
#     if i==0:
#         count_di = left_most_keyword(agg_proto_df_dict) 
#         i+=1
#     else:
    word,j,count_di = left_most_keyword(left_proto_df_dict)      
    if word=='DONE':
        break
    k+=1
#     if len(count_di)<1: #沒字了
#         break
#     word =  list(count_di.keys())[0]
    words.append(word)
    print('ROUND:',k,'take:',word,list(count_di.values())[j])
    for i,proto in enumerate(protocols_need):    
        proto_df = left_proto_df_dict[proto]
        proto_df['taken'] = proto_df['isp_date_sessioncount'].apply(is_intaken,args=(word,))
        not_take_df = proto_df[proto_df.taken==False]
        left_proto_df_dict[proto]=not_take_df
not_take_df

ROUND: 1 take: 台灣之星_20200106 314


/home/antslab/.local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/antslab/.local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/antslab/.local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

ROUND: 2 take: 台灣大哥大_20200110 254
ROUND: 3 take: 中嘉寬頻_20200106 199
ROUND: 4 take: 台灣碩網_20200107 121
ROUND: 5 take: 中嘉寬頻_20200110 94
ROUND: 6 take: 台灣碩網_20200109 80
ROUND: 7 take: 台灣大哥大_20200107 76
ROUND: 8 take: 台灣大哥大_20200109 57
ROUND: 9 take: 台灣碩網_20200111 52
ROUND: 10 take: 中嘉寬頻_20200109 46
ROUND: 11 take: 凱擘_20200109 40
ROUND: 12 take: 台灣大哥大_20200112 33
ROUND: 13 take: 台灣固網_20200107 32
ROUND: 14 take: 台灣基礎開發_20200107 28
ROUND: 15 take: 台灣碩網_20200110 26
ROUND: 16 take: 中嘉寬頻_20200111 26
ROUND: 17 take: 亞太電信_20200106 24
ROUND: 18 take: 中嘉寬頻_20200108 23
ROUND: 19 take: 台灣基礎開發_20200106 20
ROUND: 20 take: 中嘉寬頻_20200107 19
ROUND: 21 take: 台灣碩網_20200112 16
ROUND: 22 take: 台灣之星_20200108 14
ROUND: 23 take: 凱擘_20200111 14
ROUND: 24 take: 台灣大哥大_20200106 14
ROUND: 25 take: 亞太電信_20200109 13
ROUND: 26 take: 凱擘_20200106 13
ROUND: 27 take: 台灣固網_20200109 12
ROUND: 28 take: 台固媒體_20200107 11
ROUND: 29 take: 台灣之星_20200110 10
ROUND: 30 take: 台灣碩網_20200106 10
ROUND: 31 take: 台灣大哥大_20200111 10
ROUND: 32 t

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,ssdeep_clean,signature,clusterkey_timelist,date_count,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,taken
3,20200107_137438954173_imap,"[1578387079.427306, 1578387826.277441, 1578388...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...",[3:OXXUT8UU1V1V5WCX4V1A9mfVPc+fVE:O0TJCX4Q9mff...,"[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,5,[3:OXXUT8UU1V1V5WCX4V1A9mfVPc+fVE:O0TJCX4Q9mff...,[3:OXXUT8UU1V1V5WCX4V1A9mfVPc+fVE:O0TJCX4Q9mffm],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578387079.427306, 1578387079.427589, 1578387...",{'20200107': 5},{'遠傳電信_20200107': 5},331,5,1,1,False
4,20200107_28903_imap,"[1578387083.971583, 1578387833.336951, 1578388...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...",[48:rvD/IH5W0q0WKRAR2Tt9rZi8DCNnvD/CCgvi9cLMlQ...,"[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,4,[48:rvD/IH5W0q0WKRAR2Tt9rZi8DCNnvD/CCgvi9cLMlQ...,[48:rvD/IH5W0q0WKRAR2Tt9rZi8DCNnvD/CCgvi9cLMlQ...,{'48': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafzno...,"[1578388554.929654, 1578388554.929898, 1578388...",{'20200107': 4},{'遠傳電信_20200107': 4},265,4,1,1,False
5,20200107_8589954530_imap,"[1578387053.18218, 1578387788.636685, 15783883...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...",[24:XSwa7ukPW9IHdlUX18YHOSV9afC24luNiDQ2lK8AfF...,"[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,4,[24:XSwa7ukPW9IHdlUX18YHOSV9afC24luNiDQ2lK8AfF...,[24:XSwa7ukPW9IHdlUX18YHOSV9afC24luNiDQ2lK8AfF...,"{'24': ['k', 'P', 'IHdlUX18YHOSV9afC24luNiDQ2l...","[1578387788.636685, 1578387788.636935, 1578387...",{'20200107': 4},{'遠傳電信_20200107': 4},265,4,1,1,False
6,20200107_25769832945_imap,"[1578386929.380142, 1578387644.568226, 1578388...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...","[3:gfdX9Q7DMjfu:gF64jm, 3:gfdX9Q7DMjfu:gF64jm,...","[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,4,"[3:gfdX9Q7DMjfu:gF64jm, 3:gfdX9Q7DMjfu:gF64jm,...",[3:gfdX9Q7DMjfu:gF64jm],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578388389.794902, 1578388389.795204, 1578388...",{'20200107': 4},{'遠傳電信_20200107': 4},265,4,1,1,False
7,20200107_85899374904_imap,"[1578387049.159031, 1578387778.500947, 1578388...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...","[3:RfY1V1VOdC9W:5Y+C4, 3:RfY1V1VOdC9W:5Y+C4, 3...","[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,4,"[3:RfY1V1VOdC9W:5Y+C4, 3:RfY1V1VOdC9W:5Y+C4, 3...",[3:RfY1V1VOdC9W:5Y+C4],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578387778.500947, 1578387778.501239, 1578387...",{'20200107': 4},{'遠傳電信_20200107': 4},265,4,1,1,False
8,20200107_180388627132_imap,"[1578387045.110211, 1578387771.408819, 1578388...","[United States, United States, United States, ...","[171.67.70.88, 171.67.70.88, 171.67.70.88, 171...",[24:QEAW2oXy1LGN1CJnIPu4gYCae4El7Y32b6MymWQxmN...,"[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,4,[24:QEAW2oXy1LGN1CJnIPu4gYCae4El7Y32b6MymWQxmN...,[24:QEAW2oXy1LGN1CJnIPu4gYCae4El7Y32b6MymWQxmN...,{'24': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafzno...,"[1578388499.997991, 1578388499.998357, 1578388...",{'20200107': 4},{'遠傳電信_20200107': 4},265,4,1,1,False
9,20200107_25769823302_imap,"[1578387007.314452, 1578387738.63955, 15783883...","[United States, United States, United States]","[171.67.70.88, 171.67.70.88, 171.67.70.88]","[3:QVE:Qu, 3:QVE:Qu, 3:QVE:Qu]","[(37.4446, -122.1835), (37.4446, -122.1835), (...",遠傳電信,imap,3,"[3:QVE:Qu, 3:QVE:Qu, 3:QVE:Qu]",[3:QVE:Qu],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578387738.63955, 1578387738.639814, 15783877...",{'20200107': 3},{'遠傳電信_20200107': 3},199,3,1,1,False
10,20200107_25769832946_imap,"[15

In [5]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
whole_words = []
for isp in isp_li:
    for date in date_li:
        whole_words.append(isp+'_'+date)
rest = list(set(whole_words)-set(words))
words.extend(rest)
words.extend(['遠傳電信_20200107','遠傳電信_20200106'])
words

['台灣之星_20200106',
 '台灣大哥大_20200110',
 '中嘉寬頻_20200106',
 '台灣碩網_20200107',
 '中嘉寬頻_20200110',
 '台灣碩網_20200109',
 '台灣大哥大_20200107',
 '台灣大哥大_20200109',
 '台灣碩網_20200111',
 '中嘉寬頻_20200109',
 '凱擘_20200109',
 '台灣大哥大_20200112',
 '台灣固網_20200107',
 '台灣基礎開發_20200107',
 '台灣碩網_20200110',
 '中嘉寬頻_20200111',
 '亞太電信_20200106',
 '中嘉寬頻_20200108',
 '台灣基礎開發_20200106',
 '中嘉寬頻_20200107',
 '台灣碩網_20200112',
 '台灣之星_20200108',
 '凱擘_20200111',
 '台灣大哥大_20200106',
 '亞太電信_20200109',
 '凱擘_20200106',
 '台灣固網_20200109',
 '台固媒體_20200107',
 '台灣之星_20200110',
 '台灣碩網_20200106',
 '台灣大哥大_20200111',
 '中嘉寬頻_20200112',
 '台灣碩網_20200108',
 '亞太電信_20200107',
 '台灣固網_20200111',
 '凱擘_20200107',
 '台灣之星_20200107',
 '台固媒體_20200111',
 '台灣固網_20200112',
 '中華電信_20200109',
 '亞太電信_20200108',
 '亞太電信_20200112',
 '台灣基礎開發_20200110',
 '台灣之星_20200111',
 '台灣固網_20200106',
 '凱擘_20200110',
 '台固媒體_20200108',
 '亞太電信_20200111',
 '台灣基礎開發_20200108',
 '凱擘_20200108',
 '中華電信_20200106',
 '台灣基礎開發_20200111',
 '台灣基礎開發_20200109',
 '台固媒體_20200106',
 '台固媒體_20200109',
 '台灣

In [ ]:
def flat_the_lili(liolili):
    if type(liolili[0])==list:
        return list(functools.reduce(operator.iconcat, liolili, []))
    else:
        return liolili
def extract_pkttime(timestamp_li,proto_sessdf,ori_pkt_timeli,word):
    proto_sessdf = proto_sessdf[proto_sessdf['session_time'].isin(timestamp_li)]
    if (len(proto_sessdf)<1) or (len(proto_sessdf)<=len(ori_pkt_timeli[1])): #此AP無所屬的start time在這個word當中，或是已經有過了
        return ori_pkt_timeli #太短的也不拿
    else:
        kk = {}
        kk[word]=proto_sessdf.session_time_list.tolist() #
        sampled_timelist = kk #dict: 2D list : sessions' time
        sampled_sess_duration = proto_sessdf.session_duration.tolist() #1D list
        sampled_i_tt_pkts = proto_sessdf.session_i_tt_packet.tolist() #1D list
        sampled_i_tt_payload_length = proto_sessdf.tcp_i_tt_payload_length.tolist() #1D list
        sampled_i_avg_payload_length = proto_sessdf.tcp_i_avg_payload_length.tolist() #1D list
        sampled_i_payloadinfo = proto_sessdf.tcp_i_payload_list.tolist() #3D list: ssdeep,time,length
        return [sampled_timelist,sampled_sess_duration,sampled_i_tt_pkts,sampled_i_tt_payload_length,
               sampled_i_avg_payload_length,sampled_i_payloadinfo]

info_proto_df_dict = agg_proto_df_dict.copy()    
taken_isp_da = words#[:10]
for i,word in tqdm(enumerate(taken_isp_da)):
    date = word.split('_')[-1]
    date = date[:4]+'_'+date[4:6]+'_'+date[6:]
    isp = word.split('_')[0]
    path_intermeidate = '/home/antslab/NAS1_RAID6/pcap_inter/' + date + '/' + isp + '/intermeidate_data/'
    gc.collect()
    session_df,_,_ = pd.read_pickle(path_intermeidate + 'df2_tuples_' + isp + '.pkl')
    for proto in protocols_need:
        proto_df = info_proto_df_dict[proto]      
        proto_df['flat_timestamp'] = proto_df['timestamp'].apply(flat_the_lili)
        if i==0:
            temp = ['','']
            proto_df['pkt_allinfo'] = proto_df.apply(lambda x: temp,axis=1)  #initialize        
        proto_df['pkt_allinfo'] = proto_df.apply(lambda x:extract_pkttime(x.flat_timestamp,session_df,x.pkt_allinfo,word),axis=1)  
#         proto_df['pkt_allinfo'] = proto_df['flat_timestamp'].apply(args=(session_df,))
        info_proto_df_dict[proto] = proto_df
    pickle.dump(file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/crossisp_statistics_dfdict_sampled_pktinfo_123.pkl','wb'),obj=info_proto_df_dict)
#         startime_list = proto_df.flat_timestamp.tolist()
#         startime_list = list(functools.reduce(operator.iconcat,startime_list,[]))
#         proto_df['pkt_timelist'] = proto_sessdf.session_time_list.apply()
#     session_time_li = 
#         break
#     break
info_proto_df_dict['http']

23it [46:23:05, 16431.49s/it]

In [100]:
proto_df[proto_df.isp=='台灣之星']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,...,clusterkey_timelist,date_count,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,taken,pkt_allinfo,flat_timestamp
47,20200108_2_ssh,"[1578412229.980322, 1578412230.00332, 15784122...","[Ireland, Ireland, Russia, Ireland, Ireland, R...","[5.188.87.51, 5.188.86.221, 5.182.39.97, 5.188...",[3:PfRDBfRV+dCfDEflfGL73coTTnrySTMuZGIUpdSIcfE...,"[(51.8472, -9.2304), (51.9, -8.95), (55.7386, ...",台灣之星,ssh,1994475,[3:PfRDBfRV+dCfDEflfGL73coTTnrySTMuZGIUpdSIcfE...,...,"[1578460395.681916, 1578460395.682197, 1578460...","{'20200107': 14190, '20200108': 1980285}","{'台灣之星_20200107': 14190, '台灣之星_20200108': 1980...",73092273,1994475,20778,160,False,"[, ]","[1578412229.980322, 1578412230.00332, 15784122..."
48,20200107_3_ssh,"[1578325831.059465, 1578325831.128921, 1578325...","[Ireland, China, Germany, Ireland, Ireland, Ir...","[5.188.86.168, 60.29.241.2, 88.214.26.92, 5.18...",[3:jbSF5d6BrDifDpAjN98xHDfsBuGpB+WfstzAskfTp05...,"[(51.9, -8.95), (39.1423, 117.1726), (51.2993,...",台灣之星,ssh,1968637,[3:jbSF5d6BrDifDpAjN98xHDfsBuGpB+WfstzAskfTp05...,...,"[1578373394.667016, 1578373394.667192, 1578373...","{'20200106': 12443, '20200107': 1956194}","{'台灣之星_20200106': 12443, '台灣之星_20200107': 1956...",71300521,1968637,20946,158,True,"[, ]","[1578325831.059465, 1578325831.128921, 1578325..."
49,20200108_21_ssh,"[1578412230.02106, 1578412265.077625, 15784122...","[Vietnam, Vietnam, China, Vietnam, Taiwan, Vie...","[123.21.152.48, 113.172.234.217, 121.18.68.139...",[3:afNV96IKKFfVYGdyfI4QCkE6fzUc/p5VffHRGb9Efdf...,"[(10.8142, 106.6438), (10.8142, 106.6438), (38...",台灣之星,ssh,6865,[3:afNV96IKKFfVYGdyfI4QCkE6fzUc/p5VffHRGb9Efdf...,...,"[1578490867.914832, 1578490868.916248, 1578490...","{'20200107': 83, '20200108': 6782}","{'台灣之星_20200107': 83, '台灣之星_20200108': 6782}",251569,6865,1919,50,False,"[, ]","[1578412230.02106, 1578412265.077625, 15784122..."
50,20200109_515_ssh,"[1578498777.446883, 1578498821.05141, 15784988...","[Latvia, Taiwan, Vietnam, Egypt, Vietnam, Egyp...","[195.3.147.47, 113.61.128.75, 123.20.187.108, ...",[3:11VZfFgW+fmE5BRACHfWdt0fmco1fAWApfHP9fc7iLu...,"[(57.0, 25.0), (24.1469, 120.6839), (10.8142, ...",台灣之星,ssh,1462,[3:11VZfFgW+fmE5BRACHfWdt0fmco1fAWApfHP9fc7iLu...,...,"[1578525628.081056, 1578525628.081249, 1578525...","{'20200108': 13, '20200109': 1449}","{'台灣之星_20200108': 13, '台灣之星_20200109': 1449}",54976,1462,1257,52,False,"[, ]","[1578498777.446883, 1578498821.05141, 15784988..."
51,20200112_245_ssh,"[1578757954.437267, 1578757958.292281, 1578758...","[Vietnam, China, Thailand, Thailand, Egypt, Vi...","[123.21.208.105, 112.28.9.51, 183.89.154.28, 1...",[6:nZfZk+NLQ9cBoeLSxUR8FUObE1djdTffLEvqh:n7k+N...,"[(10.8142, 106.6438), (31.8642, 117.2865), (13...",台灣之星,ssh,1104,[6:nZfZk+NLQ9cBoeLSxUR8FUObE1djdTffLEvqh:n7k+N...,...,"[1578814764.161106, 1578814764.161368, 1578814...","{'20200111': 11, '20200112': 1093}","{'台灣之星_20200111': 11, '台灣之星_20200112': 1093}",45950,1104,954,50,False,"[, ]","[1578757954.437267, 1578757958.292281, 1578758..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,20200108_541165888518_ssh,"[1578412248.588297, 1578431989.678822]","[Ireland, Netherlands]","[5.188.86.168, 190.2.153.12]",[3:pGCYfBr+bDfXS1TWKNpLpDcXPGEfOafTvU9XEPkBqWc...,"[(51.9, -8.95), (51.9935, 4.207)]",台灣之星,ssh,2,[3:pGCYfBr+bDfXS1TWKNpLpDcXPGEfOafTvU9XEPkBqWc...,...,"[1578431989.678822, 1578431989.679062, 1578431...","{'20200107': 1, '20200108': 1}","{'台灣之星_20200107': 1, '台灣之星_20200108': 1}",72,2,2,2,False,"[, ]","[1578412248.588297, 1578431989.678822]"
311,20200109_670014914792_ssh,"[1578498633.93722, 1578552878.173917]","[Ireland, Netherlands]","[5.188.86.168, 190.2.153.12]",[3:PuddQfhf2cfdOeyfHCLfISOv5MubcwpPSfU3A9UMgrP...,"[(51.9, -8.95), (51.9935, 4.207)]",台灣之星,ssh,2,[3:PuddQfhf2cfdOeyfHCLfISOv5MubcwpPSfU3A9UMgrP...,...,"[1578552878.173917, 1578552878.174094, 1578552...","{'202001

In [103]:
session_df[session_df.session_time==1578325831.128921]

,session_time,session_time_list,session_duration,session_i_tt_packet,session_o_tt_packet,session_i_tt_frame_length,session_o_tt_frame_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,...,ip_dst,tcp_srcport,tcp_dstport,country,isp,domain,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list


In [98]:
proto_df[proto_df['pkt_allinfo'].apply(len)<3],word

(                                                    idx  \
 0                                    20200108_11264_ssh   
 1                               20200106_8589977436_ssh   
 2                                     20200108_9975_ssh   
 3                             20200106_128849057833_ssh   
 4                               20200106_8589981055_ssh   
 ...                                                 ...   
 1013  20200107_1202590842991_ssh_20200107_3264175200...   
 1014  20200112_25769852139_ssh_20200112_103079226446...   
 1015  20200112_240518181600_ssh_20200112_60129542375...   
 1016       20200108_154618825525_ssh_20200110_10005_ssh   
 1017  20200107_111669169414_ssh_20200107_29205779578...   
 
                                               timestamp  \
 0     [1578412225.050548, 1578412225.134294, 1578412...   
 1     [1578241179.402656, 1578245035.340037, 1578249...   
 2     [1578452582.177188, 1578452582.188534, 1578452...   
 3     [1578292014.186076, 1578292511.

In [83]:
session_df.tcp_i_payload_list[3]

[['3:DfQMPmfK7PATOwWGX6z:DPQK7wG', '1578309184.707190000', '23'],
 ['48:dbUMAM8FMyFMdGo2MZZJjI2ewelxreuD2MZZJjI2ewelxreu/ZMCMP63tSt7ZMCR:d3/HphZslw6ZslwGgDgFS',
  '1578309184.809852000',
  '744'],
 ['3:11URZXX3vV5vfV1VTV1VVfV1Vd/ufWX:kRZXX', '1578309184.927153000', '24'],
 ['12:JVTNJ6A+uw/pIrHffm+qAxAQX0DW98q3zHZd7RLtWIwd/Tdvj8:JVTiFGfujWmyH33WIILZo',
  '1578309185.009044000',
  '208'],
 ['3:11VZfC4f1HRfdT7+:nzf1ZdTK', '1578309185.128981000', '16'],
 ['3:Avu5lrSfGwhfUTX7KaSo1QVVfaZfXBfS1cNfoFP4APESpzeVsaq+Vfgb:Avu5YfriuoaV14o1cNfot4MlzeVspqfC',
  '1578309185.198063000',
  '52'],
 ['6:2e1iU3NvDi+YcbpaSMOelxDhXs0WWwkN0PtOEP8f2H9KMEfDvB:H1iU3hDWc2OelxDhFvOAG8XDvB',
  '1578309185.309564000',
  '100'],
 ['6:i2T5BWfFw4KVdGyvV5QDmyqYTjEYp+BfDTqcdyMupAjy1Dn:6wrdGyvV5wmG/2fDjYxujy1j',
  '1578309186.412370000',
  '84']]

In [74]:

proto_df = agg_proto_df_dict['http']
proto_df['flat_timestamp'] = proto_df['timestamp'].apply(flat_the_lili)
startime_list = proto_df.flat_timestamp.tolist()
startime_list = list(functools.reduce(operator.iconcat,startime_list,[]))
startime_list

[1578491422.648805,
 1578491434.488182,
 1578491490.187107,
 1578491494.303335,
 1578491496.971089,
 1578491500.667118,
 1578491522.02379,
 1578553728.922476,
 1578553736.509628,
 1578553740.377186,
 1578553745.276913,
 1578553747.09236,
 1578553747.674099,
 1578553802.182323,
 1578553803.745613,
 1578553807.915182,
 1578553809.645606,
 1578553835.834429,
 1578553841.279505,
 1578553841.367504,
 1578553842.621844,
 1578553847.842346,
 1578553850.088855,
 1578553851.955171,
 1578553853.556296,
 1578553874.393467,
 1578553874.749563,
 1578553883.699246,
 1578553921.753201,
 1578553937.222146,
 1578557654.211719,
 1578557663.126458,
 1578557666.039217,
 1578557671.422898,
 1578557675.069997,
 1578557681.183768,
 1578557685.079858,
 1578557688.057023,
 1578557724.030381,
 1578557726.363397,
 1578557751.878363,
 1578557758.148478,
 1578557759.742256,
 1578557760.398175,
 1578557771.464541,
 1578557771.69092,
 1578557772.732711,
 1578557772.805718,
 1578557773.554555,
 1578557774.667835,
 15

In [75]:
startime_list[-1]

1578748048.624227

In [69]:
proto_df

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,...,ssdeep_clean,clusterkey_timelist,date_count,sum,isp_date_sessioncount,total_packets_num,total_sessions_num,total_ip_num,total_country_num,taken
0,20200108_8589990159_http,"[1578491422.648805, 1578491434.488182, 1578491...","[South Korea, South Korea, South Korea, South ...","[122.41.87.188, 122.41.87.188, 122.41.87.188, ...",[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,"[(37.5112, 126.9741), (37.5112, 126.9741), (37...",亞太電信,http,118,[48:o0SP/FgTwfTihvZMlsSp0UUwaIqF6GY18nR:PSPdMw...,...,[48:Ga7UCqF6GY18nwiLkcRq09jcRq09uMRRdJtYMY8:hg...,"[1578491500.667118, 1578491500.667435, 1578491...","{'20200108': 7, '20200109': 111}",0.0,"{'亞太電信_20200108': 7, '亞太電信_20200109': 111}",21036,118,3,3,False
1,20200106_8589962872_http,"[1578244875.871224, 1578279973.587717, 1578291...","[Netherlands, Netherlands, Netherlands, Nether...","[51.15.71.134, 51.15.71.134, 51.15.71.134, 51....",[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,"[(52.3759, 4.8975), (52.3759, 4.8975), (52.375...",亞太電信,http,25,[24:LepsAkMVAab5CYD0sVUYaP+shmGvQEjyOFEoW:Leps...,...,[12:Bhzeps+hkMVk3HXq6xgzhrHCYPEjyOFE4nBQom:Lep...,"[1578315077.277642, 1578315077.277914, 1578315...","{'20200106': 5, '20200107': 5, '20200108': 2, ...",0.0,"{'亞太電信_20200106': 5, '亞太電信_20200107': 5, '亞太電信...",5434,25,1,1,False
2,20200106_206158498684_http,"[1578325476.581104, 1578325480.685079]","[Canada, Canada]","[209.99.129.253, 209.99.129.253]",[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[(43.6655, -79.4204), (43.6655, -79.4204)]",亞太電信,http,2,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,...,[96:rH/x3HHVT/TpXneNXZNJ2xNXJNf+QDsAbQ41uw10p8...,"[1578325476.581104, 1578325476.581368, 1578325...",{'20200106': 2},0.0,{'亞太電信_20200106': 2},441,2,1,1,False
3,20200109_128849077624_http,"[1578544003.66182, 1578544004.619287]","[United Kingdom, United Kingdom]","[37.220.22.180, 88.150.210.33]",[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[(53.552, -1.4872), (51.1912, -0.3432)]",亞太電信,http,2,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,...,[48:Rfxa7UXe4CyGXU9faY18nOtRwXA4AfXbmW:R2tdyGX...,"[1578544004.619287, 1578544004.619586, 1578544...",{'20200109': 2},0.0,{'亞太電信_20200109': 2},346,2,2,1,False
4,20200106_534_http,"[1578244070.453082, 1578248759.696275, 1578252...","[China, France, Thailand, Thailand, China, Ban...","[119.52.109.64, 83.206.128.205, 146.88.51.234,...",[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,"[(42.5272, 125.6753), (48.7142, 7.1898), (13.7...",中華電信,http,36,[96:ttUMItUJ3FNnXFFYFF3Ib4su4WnXFIb4sv46Ib4sB4...,...,[96:TnXFUYU33ImnXFI1I4PUmlXnXNUm4dUm8q3UUKnXFU...,"[1578295954.030631, 1578295954.030895, 1578295...",{'20200106': 36},0.0,{'中華電信_20200106': 36},31210,36,12,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,20200109_317827601065_http_20200109_1632087680...,"[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,...,[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854...","{'20200109': 8, '20200110': 3}",0.0,"{'台灣碩網_20200109': 8, '台灣碩網_20200110': 3}",15332,11,2,2,False
340,20200107_206158449329_http_20200106_1202590851...,"[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386, 37.6068), (40.71...","[台灣碩網, 台灣碩網, 台灣碩網]","[http, http, http]","[5, 3, 3]",[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5B...,...,[96:7+y28P2BI9XyS1uw1sO5Yy28P2BI9Xyn5Kx:7+yX2B...,"[[1578333098.110814, 1578333098.111041, 157833...","{'2020010

In [56]:
date = taken_isp_da[0].split('_')[-1]
date = date[:4]+'_'+date[4:6]+'_'+date[6:]
isp = taken_isp_da[0].split('_')[0]